### Разметка твитов Илона Маска

Ссылки

https://www.kaggle.com/code/buyuknacar/comparing-sentiment-classifiers-txblob-vader-bert

https://www.kaggle.com/discussions/questions-and-answers/206813

https://huggingface.co/search/full-text?q=sentiment+twitter+analysis+english&type=model

some models

https://huggingface.co/agentlans/deberta-v3-xsmall-tweet-sentiment

https://huggingface.co/Manas2708/bert-base-sentiment-analysis

https://huggingface.co/siebert/sentiment-roberta-large-english

https://huggingface.co/j-hartmann/emotion-english-distilroberta-base

https://huggingface.co/michellejieli/emotion_text_classifier

https://huggingface.co/Linna/emotion-english-distilroberta-melinna

https://huggingface.co/mrhacker7599/emotion-english-distilroberta-base

https://huggingface.co/MicahB/emotion_text_classifier

https://huggingface.co/pysentimiento/robertuito-sentiment-analysis

https://huggingface.co/pysentimiento/bertweet-pt-sentiment

https://huggingface.co/pysentimiento/robertuito-emotion-analysis

In [1]:
# !pip install textblob
from textblob import TextBlob
import pandas as pd

In [2]:
from textblob.sentiments import NaiveBayesAnalyzer

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Тестируем

In [4]:
df = pd.read_parquet('df.parquet')

1

In [5]:
analyzer = SentimentIntensityAnalyzer()

In [6]:
def return_label(sent):
    scores = analyzer.polarity_scores(sent)
    pos, neg = scores.get('pos'), scores.get('neg')
    if pos > neg:
        return 1
    else:
        return 0

In [7]:
%%time
df['feeling_auto'] = df['cleaned_tweet'].apply(lambda x : return_label(x))

CPU times: user 3.3 s, sys: 13.1 ms, total: 3.32 s
Wall time: 3.32 s


По итогу 30% текста оказалось размечено иначе:

In [8]:
(df.feeling != df.feeling_auto).mean()

0.34744141080774743

2

In [9]:
%%time
df['feeling_auto'] = df['cleaned_tweet'].apply(lambda x : TextBlob(x).sentiment.polarity)

CPU times: user 7.16 s, sys: 48.4 ms, total: 7.2 s
Wall time: 7.21 s


In [10]:
df.feeling_auto = df.feeling_auto.apply(lambda x : 0 if x < 0 else 1)

По итогу 40% текста оказалось размечено иначе:

In [11]:
(df.feeling != df.feeling_auto).mean()

0.3929126245957929

3

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

In [10]:
%%time

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Error while downloading from https://cdn-lfs.hf.co/nlptown/bert-base-multilingual-uncased-sentiment/e72c9084bf53cee1f722b1acc6f7439cce27e9beabddce447653a1af45405183?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1733439397&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMzQzOTM5N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9ubHB0b3duL2JlcnQtYmFzZS1tdWx0aWxpbmd1YWwtdW5jYXNlZC1zZW50aW1lbnQvZTcyYzkwODRiZjUzY2VlMWY3MjJiMWFjYzZmNzQzOWNjZTI3ZTliZWFiZGRjZTQ0NzY1M2ExYWY0NTQwNTE4Mz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV19&Signature=he2V%7E4-60mIrWUoOveo6GDL-hurzK54ogBtDQmLw9h8r0Wytr8g3jbZCWIuJ3UAH9x2GYNsy0CVp1%7ELiCTGI4KOSIkyNAJry2FmP1HSbAzNq1s06Xc9qaiX%7EFcmcg9mhrE6gVklvn%7E-oqvTI4oJhJixu44Ahn1cBvpqvmfXQroQB4aRTyDm12HpnmvF9dGH%7E8o2TrFKJMYuzAdqDvkwbxP1pYBe6Wdd1rKNWsNbHlJBUoYAESoiB%7

CPU times: user 1.54 s, sys: 3.12 s, total: 4.66 s
Wall time: 7min 5s


In [13]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [27]:
%%time
df['feeling_auto'] = df.cleaned_tweet.apply(lambda x : sentiment_score(x) >= 3)

CPU times: user 6h 36min 25s, sys: 55min 53s, total: 7h 32min 18s
Wall time: 1h 54min 1s


In [29]:
(df.feeling != df.feeling_auto).mean()

0.3145514551455145

4

In [32]:
from transformers import pipeline

In [33]:
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [44]:
%%time
df['feeling_auto'] = df.cleaned_tweet.apply(lambda x : 1 if sentiment_analysis(x)[0].get('label') == 'POSITIVE' else 0)

CPU times: user 20h 5min 58s, sys: 1h 50min 35s, total: 21h 56min 33s
Wall time: 5h 30min 59s


In [45]:
(df.feeling != df.feeling_auto).mean()

0.24835150181684834

In [47]:
df.to_parquet('df_feeling_auto.parquet')

In [52]:
df.loc[df.feeling != df.feeling_auto, ['cleaned_tweet', 'feeling', 'feeling_auto']].cleaned_tweet.values[5]

'get over it jk im going to miss you'

In [65]:
df.loc[df.feeling != df.feeling_auto, ['cleaned_tweet', 'feeling', 'feeling_auto']].iloc[51]

cleaned_tweet    so happy cuz u talked with my bff gosh i misse...
feeling                                                          0
feeling_auto                                                     1
Name: 187, dtype: object

In [60]:
df.loc[df.feeling != df.feeling_auto, ['cleaned_tweet', 'feeling', 'feeling_auto']].iloc[45]

cleaned_tweet    why
feeling            0
feeling_auto       1
Name: 155, dtype: object

In [67]:
df.loc[df.feeling != df.feeling_auto, ['cleaned_tweet', 'feeling', 'feeling_auto']].iloc[87]

cleaned_tweet    nothing i was doing my biologys work and now i...
feeling                                                          1
feeling_auto                                                     0
Name: 299, dtype: object

In [71]:
df.loc[df.feeling != df.feeling_auto, ['cleaned_tweet', 'feeling', 'feeling_auto']].iloc[99]

cleaned_tweet    sorry to miss your textlooks like your evening...
feeling                                                          1
feeling_auto                                                     0
Name: 346, dtype: object

In [74]:
df.loc[df.feeling != df.feeling_auto, ['cleaned_tweet', 'feeling', 'feeling_auto']].iloc[1555]

cleaned_tweet    playing wit my cuzs i just started waling on m...
feeling                                                          1
feeling_auto                                                     0
Name: 6164, dtype: object

**Валидируем без выбросов**

In [1]:
# !pip install textblob
from textblob import TextBlob
import pandas as pd

In [20]:
df = pd.read_parquet('df_feeling_auto.parquet').loc[:, ['cleaned_tweet', 'feeling_auto', 'feeling']]

In [21]:
df

,cleaned_tweet,feeling_auto,feeling
0,starting back at work today looks like itll be...,0,0
1,sugar levels dropping munchies setting in luck...,1,1
2,yeah have a great summer break,1,1
3,hannah montana was very good now going to read...,1,1
4,aww have fun i just had my d last week,1,1
...,...,...,...
149980,had a great night at tabs but now im missing c...,0,0
149981,what a night at work church in the morning,1,1
149982,this book is terrible im used reading all make...,0,0
149983,updates any i miss your updates guys,1,0


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
tfidf_vectorizer = TfidfVectorizer(max_features=100)

In [6]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df.cleaned_tweet).toarray()

In [7]:
from sklearn.manifold import TSNE

In [8]:
%%time
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=300)
tsne_result_tfidf = tsne.fit_transform(tfidf_matrix)

CPU times: user 33min 21s, sys: 1min 57s, total: 35min 19s
Wall time: 4min 35s


In [41]:
tsne_df = pd.DataFrame(tsne_result_tfidf, columns=['Component 1', 'Component 2'])
tsne_df

,Component 1,Component 2
0,5.640377,4.015611
1,-5.851907,3.167936
2,-0.190767,-0.220425
3,3.871474,2.504313
4,8.264714,1.663213
...,...,...
149980,7.838943,2.908100
149981,9.353568,2.400236
149982,-2.539329,-1.242003
149983,3.201050,-9.481975


In [42]:
import numpy as np

In [43]:
outliers_condition1 = (tsne_df['Component 1'] <= tsne_df['Component 1'].quantile(0.97))
outliers_condition2 = (tsne_df['Component 1'] >= tsne_df['Component 1'].quantile(0.01))
outliers_condition3 = (tsne_df['Component 2'] <= tsne_df['Component 2'].quantile(0.99))
outliers_condition4 = (tsne_df['Component 2'] >= tsne_df['Component 2'].quantile(0.01))
not_outliers = np.all([outliers_condition1, outliers_condition2, outliers_condition3, outliers_condition4], axis=0)
tsne_df = tsne_df.loc[not_outliers, :]

In [48]:
df2 = pd.concat([tsne_df, df.loc[not_outliers, ['feeling_auto', 'feeling', 'cleaned_tweet']]], axis=1)

In [76]:
df3 = df2.loc[(df2.feeling != df2.feeling_auto)].iloc[:,2:].sample(50)

In [110]:
df3.columns, df3.iloc[-15].values

(Index(['feeling_auto', 'feeling', 'cleaned_tweet'], dtype='object'),
 array([0, 1, 'grrrr i hate subo cmon hollie'], dtype=object))

**Разметка Илона Маска**

In [111]:
df = pd.read_csv('elon_musk_tweets.csv')

In [113]:
df.text

0       @BillyM2k I find the gold toe sock – inevitabl...
1                      Sock Con, the conference for socks
2       Always something new for the magazine cover an...
3                        @ExplainThisBob This guy gets it
4       Sock tech is so advanced that you can get pret...
                              ...                        
5899                       @JonErlichman He’s not wrong …
5900    @alifarhat79 Guys, I think I maybe took too mu...
5901                                        @sriramk Cool
5902                 @cb_doge Time to complete the circle
5903           @Jason Late stage civilization complacency
Name: text, Length: 5904, dtype: object

In [114]:
from transformers import pipeline

/opt/homebrew/Caskroom/miniconda/base/envs/course-work-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [115]:
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [116]:
%%time
df['feeling_auto'] = df.text.apply(lambda x : 1 if sentiment_analysis(x)[0].get('label') == 'POSITIVE' else 0)

CPU times: user 46min 7s, sys: 4min 39s, total: 50min 46s
Wall time: 12min 44s


In [118]:
df.to_parquet('elon_musk_tweets_labeled.parquet')

**csv**

In [1]:
import pandas as pd